# WW resummation

In [1]:
import os, sys, pickle, copy
import datetime
sys.path.append(os.getcwd() + "/../")
from prep_ana_II import *
from cross_section_calc import calc_cross_stuff, cross_calc, stat_unc_calc, normalization_unc_calc
import warnings
warnings.filterwarnings("ignore")

Welcome to JupyROOT 6.10/00
Loading binning options...
Loading plotting options
unc_mc_process and scales are parameter dictionaries


In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload

#ToDo:
#+?make those WW resummation plots like I made last year
#+?Remove the shit that makes no sense....because this comment is sooo helpful
#+?the current estimate is resummation: 0.021599299139
                           #scaled:     0.0131627437121
                           #total:      0.0252940219286

In [4]:
flavor = "both"
from sklearn.externals import joblib
df    = load_presel_w_fDY_fTT_MC()
df_da = load_presel_w_fDY_fTT_DATA()

random_forests = joblib.load("../data/RF/sep_05_fDY_fTT.jbl")
features_fDY = random_forests["features_fDY"]
clf_fDY = random_forests["clf_fDY"]
features_fTT = random_forests["features_fTT"]
clf_fTT = random_forests["clf_fTT"]

df_ww = rp.read_root(data_path+"/ww_complete.root", columns=columns)

pred_fTT = clf_fTT.predict_proba(np.float32(df_ww[features_fTT].values))
df_ww["pred_fTT_WW"] = pred_fTT[:,0]

pred_fDY = clf_fDY.predict_proba(np.float32(df_ww[features_fDY].values))
df_ww["pred_fDY_WW"] = pred_fDY[:,0]

#that_dic = pkl.load(open("../data/WWpt/8TeV_nnnlo.pkl", "r"))
resumm_weights = {}
for ft in ["normal", "np", "powheg"]:
    resumm_weights[ft + "_central"] = pd.read_csv("../data/WWpt/resummation_central_"+ft+"_weights.csv", index_col=False)
    resumm_weights[ft + "_resum_up"] = pd.read_csv("../data/WWpt/resummation_resum_up_"+ft+"_weights.csv", index_col=False )
    resumm_weights[ft + "_resum_down"] = pd.read_csv("../data/WWpt/resummation_resum_down_"+ft+"_weights.csv", index_col=False)
    resumm_weights[ft + "_scale_up"] = pd.read_csv("../data/WWpt/resummation_scale_up_"+ft+"_weights.csv", index_col=False )
    resumm_weights[ft + "_scale_down"] = pd.read_csv("../data/WWpt/resummation_scale_down_"+ft+"_weights.csv", index_col=False)
temp_df = df_ww[df_ww.ww_pt > -1]

#print "tot shape", temp_df.shape
temp_df["ww_weights"] = [1.] * temp_df.shape[0]




Warning in <TStreamerInfo::BuildCheck>: 
   The StreamerInfo of class string read from file /home/gunter/WW_analysis/data_alpha/ww_complete.root
   has the same version (=2) as the active class but a different checksum.
   You should update the version to ClassDef(string,3).
   Do not try to write objects with the current class definition,
   the files will not be readable.

Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 2 of class 'string' is missing from 
the on-file layout version 2:
   string::_Alloc_hider _M_dataplus; //
Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 2 of class 'string' is missing from 
the on-file layout version 2:
   unsigned long _M_string_length; //
Warning in <TStreamerInfo::CompareContent>: The following data member of
the in-memory layout version 2 of class 'string' is missing from 
the on-file layout version 2:
   string::(anonymous) ; //


In [7]:
orig = 0
q_ = []
r_ = []
for k in resumm_weights:
    for it, row in resumm_weights[k].iterrows():
        temp_df.ww_weights.values[ (temp_df.ww_pt >= row["low"]).values & (temp_df.ww_pt < row["high"]).values]  = row["weight"]

    rf_df = rf_ana(temp_df)
    if flavor == "diff":
        flavor_cut = rf_df.lep_Type > 0
    elif flavor == "same":
        flavor_cut = rf_df.lep_Type < 0
    else:
        flavor_cut = rf_df.lep_Type > -99

    if "central" in k:
        orig = rf_df[flavor_cut].ww_weights.sum()/temp_df.ww_weights.sum()
    #print "Weights diff correction", rf_ana(temp_df).weight.sum(), (rf_ana(temp_df).weight.values * rf_ana(temp_df).ww_weights.values).sum()
    if "resum" in k:
        r_.append(rf_df[flavor_cut].ww_weights.sum()/temp_df.ww_weights.sum())
    if "scale" in k :
        q_.append(rf_df[flavor_cut].ww_weights.sum()/temp_df.ww_weights.sum())




In [14]:
#print "orig", orig
#print "r_", r_
#print "q_", q_
r__ = np.array([abs(orig-i) for i in r_]).mean()
q__ = np.array([abs(orig-i) for i in q_]).mean()
ww_pt_unc = (q__**2 + r__**2)**.5
print r__, q__, ww_pt_unc

print cross_calc(df, df_da, df_ww, scales, fiducial=True) * ww_pt_unc





0.021599299139 0.0131627437121 0.0252940219286
0.0540619605026
